# Data preprocess

## Overview
This notebook prepares and explores demographic and real-estate CSV data (originally for Barcelona paths in the code) and sets up plotting/EDA for Paris. Key pieces of code:

1. Merging csv into multiyear "time series" data
2. Converting shapefile info of Barcelona into standard GeoJSON
3. Loading and previewing data



## Merge Barcelona info into time series

In [12]:
import pandas as pd
from pathlib import Path

def merge_csv_per_year(name, yearStart, yearEnd, input_dir, output_path=None, verbose=True):
    """
    Merge yearly files named {year}_{name}.csv for years 2015-2025
    into a single CSV named yearStart-yearEnd_{name}.csv by default.

    Parameters:
    - input_dir: path to the folder containing the yearly CSVs (string or Path)
    - output_path: optional path (string or Path) for the merged CSV file
    - verbose: print progress/messages if True

    Returns:
    - pandas.DataFrame with the concatenated data
    """
    input_dir = Path(input_dir)
    years = range(yearStart, yearEnd + 1)
    dfs = []
    for y in years:
        fp = input_dir / f"{y}_{name}.csv"
        if fp.exists():
            try:
                df = pd.read_csv(fp)
                df = df.iloc[:, 1:]     # Drop first column since it's a date 
                df['year'] = y
                dfs.append(df)
                if verbose:
                    print(f"Loaded: {fp.name} ({len(df)} rows)")
            except Exception as e:
                if verbose:
                    print(f"Failed to read {fp}: {e}")
        else:
            if verbose:
                print(f"Missing: {fp.name}")
    if not dfs:
        raise FileNotFoundError("No input files found for 2015-2025 in " + str(input_dir))
    out_df = pd.concat(dfs, ignore_index=True, sort=False)
    out_path = Path(output_path) if output_path else input_dir / f"{yearStart}-{yearEnd}_{name}.csv"
    out_df.to_csv(out_path, index=False)
    if verbose:
        print(f"Saved merged CSV to: {out_path} ({len(out_df)} total rows)")
    return out_df



In [13]:
merge_csv_per_year(name="loc_hab_valors", yearStart=2018, yearEnd=2025, input_dir="../data/raw/barcelona", output_path="../data/preprocessed/barcelona/2018-2025_loc_hab_valors.csv", verbose=True)

Loaded: 2018_loc_hab_valors.csv (6408 rows)
Loaded: 2019_loc_hab_valors.csv (6408 rows)
Loaded: 2020_loc_hab_valors.csv (6408 rows)
Loaded: 2021_loc_hab_valors.csv (6408 rows)
Loaded: 2022_loc_hab_valors.csv (6408 rows)
Loaded: 2023_loc_hab_valors.csv (6408 rows)
Loaded: 2024_loc_hab_valors.csv (6408 rows)
Loaded: 2025_loc_hab_valors.csv (6408 rows)
Saved merged CSV to: ../data/preprocessed/barcelona/2018-2025_loc_hab_valors.csv (51264 total rows)


,Codi_districte,Nom_districte,Codi_barri,Nom_barri,Seccio_censal,Desc_valors,Valors,year
0,1,Ciutat Vella,1,el Raval,1,Valor_cadastral_total_€,67059673.90,2018
1,1,Ciutat Vella,1,el Raval,2,Valor_cadastral_total_€,49981470.70,2018
2,1,Ciutat Vella,1,el Raval,3,Valor_cadastral_total_€,71692363.50,2018
3,1,Ciutat Vella,1,el Raval,4,Valor_cadastral_total_€,93534623.30,2018
4,1,Ciutat Vella,1,el Raval,5,Valor_cadastral_total_€,96442596.50,2018
...,...,...,...,...,...,...,...,...
51259,10,Sant Martí,73,la Verneda i la Pau,139,Valor_sòl_unitari_€/m2,309.26,2025
51260,10,Sant Martí,73,la Verneda i la Pau,140,Valor_sòl_unitari_€/m2,278.40,2025
51261,10,Sant Martí,73,la Verneda i la Pau,141,Valor_sòl_unitari_€/m2,291.54,2025
51262,10,Sant Martí,73,la Verneda i la Pau,142,Valor_sòl_unitari_€/m2,295.21,2025


In [14]:
merge_csv_per_year(name="pad_mdbas_niv-educa-esta_sexe", yearStart=2015, yearEnd=2025, input_dir="../data/raw/barcelona", output_path="../data/preprocessed/barcelona/2018-2025_pad_mdbas_niv-educa-esta_sexe.csv", verbose=True)

Loaded: 2015_pad_mdbas_niv-educa-esta_sexe.csv (12797 rows)
Loaded: 2016_pad_mdbas_niv-educa-esta_sexe.csv (12779 rows)
Loaded: 2017_pad_mdbas_niv-educa-esta_sexe.csv (12780 rows)
Loaded: 2018_pad_mdbas_niv-educa-esta_sexe.csv (12769 rows)
Loaded: 2019_pad_mdbas_niv-educa-esta_sexe.csv (12761 rows)
Loaded: 2020_pad_mdbas_niv-educa-esta_sexe.csv (12750 rows)
Loaded: 2021_pad_mdbas_niv-educa-esta_sexe.csv (12730 rows)
Loaded: 2022_pad_mdbas_niv-educa-esta_sexe.csv (12709 rows)
Loaded: 2023_pad_mdbas_niv-educa-esta_sexe.csv (12687 rows)
Loaded: 2024_pad_mdbas_niv-educa-esta_sexe.csv (11274 rows)
Loaded: 2025_pad_mdbas_niv-educa-esta_sexe.csv (12032 rows)
Saved merged CSV to: ../data/preprocessed/barcelona/2018-2025_pad_mdbas_niv-educa-esta_sexe.csv (138068 total rows)


,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,AEB,Seccio_Censal,Valor,NIV_EDUCA_esta,SEXE,year
0,1,Ciutat Vella,1,el Raval,1,1001,40,1,1,2015
1,1,Ciutat Vella,1,el Raval,1,1001,30,1,2,2015
2,1,Ciutat Vella,1,el Raval,1,1001,120,2,1,2015
3,1,Ciutat Vella,1,el Raval,1,1001,207,2,2,2015
4,1,Ciutat Vella,1,el Raval,1,1001,82,3,1,2015
...,...,...,...,...,...,...,...,...,...,...
138063,10,Sant Martí,73,la Verneda i la Pau,233,10143,254,3,2,2025
138064,10,Sant Martí,73,la Verneda i la Pau,233,10143,212,4,1,2025
138065,10,Sant Martí,73,la Verneda i la Pau,233,10143,211,4,2,2025
138066,10,Sant Martí,73,la Verneda i la Pau,233,10143,193,5,1,2025


In [15]:
merge_csv_per_year(name="atles_renda_bruta_llar", yearStart=2015, yearEnd=2023, input_dir="../data/raw/barcelona", output_path="../data/preprocessed/barcelona/2018-2025_atles_renda_bruta_llar.csv", verbose=True)

Loaded: 2015_atles_renda_bruta_llar.csv (1068 rows)
Loaded: 2016_atles_renda_bruta_llar.csv (1068 rows)
Loaded: 2017_atles_renda_bruta_llar.csv (1068 rows)
Loaded: 2018_atles_renda_bruta_llar.csv (1068 rows)
Loaded: 2019_atles_renda_bruta_llar.csv (1068 rows)
Loaded: 2020_atles_renda_bruta_llar.csv (1068 rows)
Loaded: 2021_atles_renda_bruta_llar.csv (1068 rows)
Loaded: 2022_atles_renda_bruta_llar.csv (1068 rows)
Loaded: 2023_atles_renda_bruta_llar.csv (1068 rows)
Saved merged CSV to: ../data/preprocessed/barcelona/2018-2025_atles_renda_bruta_llar.csv (9612 total rows)


,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Seccio_Censal,Import_Renda_Bruta_€,year
0,1,Ciutat Vella,1,el Raval,1,28573,2015
1,1,Ciutat Vella,1,el Raval,2,23673,2015
2,1,Ciutat Vella,1,el Raval,3,24997,2015
3,1,Ciutat Vella,1,el Raval,4,29768,2015
4,1,Ciutat Vella,1,el Raval,5,23645,2015
...,...,...,...,...,...,...,...
9607,10,Sant Martí,73,la Verneda i la Pau,143,44319,2023
9608,10,Sant Martí,65,el Clot,234,51993,2023
9609,10,Sant Martí,69,Diagonal Mar i el Front Marítim del Poblenou,235,40688,2023
9610,10,Sant Martí,69,Diagonal Mar i el Front Marítim del Poblenou,236,43632,2023


In [16]:
merge_csv_per_year(name="pad_mdba_sexe_edat-1", yearStart=2018, yearEnd=2025, input_dir="../data/raw/barcelona", output_path="../data/preprocessed/barcelona/2018-2025-pad_mdba_sexe_edat-1.csv", verbose=True)

Loaded: 2018_pad_mdba_sexe_edat-1.csv (45936 rows)
Loaded: 2019_pad_mdba_sexe_edat-1.csv (46006 rows)
Loaded: 2020_pad_mdba_sexe_edat-1.csv (46086 rows)
Loaded: 2021_pad_mdba_sexe_edat-1.csv (46080 rows)
Loaded: 2022_pad_mdba_sexe_edat-1.csv (46110 rows)
Loaded: 2023_pad_mdba_sexe_edat-1.csv (46149 rows)
Loaded: 2024_pad_mdba_sexe_edat-1.csv (46162 rows)
Loaded: 2025_pad_mdba_sexe_edat-1.csv (46231 rows)
Saved merged CSV to: ../data/preprocessed/barcelona/2018-2025-pad_mdba_sexe_edat-1.csv (368760 total rows)


,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,AEB,Valor,SEXE,EDAT_1,year
0,1,Ciutat Vella,1,el Raval,1,10,1,0,2018
1,1,Ciutat Vella,1,el Raval,1,12,1,1,2018
2,1,Ciutat Vella,1,el Raval,1,10,1,2,2018
3,1,Ciutat Vella,1,el Raval,1,12,1,3,2018
4,1,Ciutat Vella,1,el Raval,1,8,1,4,2018
...,...,...,...,...,...,...,...,...,...
368755,10,Sant Martí,73,la Verneda i la Pau,233,..,1,98,2025
368756,10,Sant Martí,73,la Verneda i la Pau,233,..,1,99,2025
368757,10,Sant Martí,73,la Verneda i la Pau,233,..,2,99,2025
368758,10,Sant Martí,73,la Verneda i la Pau,233,6,1,100,2025


# Export Barcelona geometric data to GeoJSON

In [17]:
import geopandas as gpd

gdf = gpd.read_file("../data/raw/barcelona/BCN_UNITATS_ADM.zip")
gdf = gdf.to_crs(epsg=4326) # epsg 4326 = WGS84 standard for lat/lon
# Save to GeoJSON
gdf.to_file("../data/preprocessed/barcelona/barcelona_neighborhoods.geojson", driver="GeoJSON")

/Users/pibe/miniconda/envs/geo/lib/python3.12/site-packages/pyogrio/geopandas.py:265: UserWarning: More than one layer found in 'BCN_UNITATS_ADM.zip': '0301040100_GranBarri_UNITATS_ADM' (default), '0301040100_SecCens_UNITATS_ADM', '0301040100_TermeMunicipal_UNITATS_ADM', '0301040100_ZUA_UNITATS_ADM', '0301040100_AEB_UNITATS_ADM', '0301040100_Àrea_I_UNITATS_ADM', '0301040100_Barris_UNITATS_ADM', '0301040100_Districtes_UNITATS_ADM'. Specify layer parameter to avoid this warning.
  result = read_func(


# Load and Inspect Preprocessed Data

In [18]:
import pandas as pd
import geopandas as gpd

# Load the datasets
try:
    property_values_df = pd.read_csv('../data/preprocessed/barcelona/2018-2025_loc_hab_valors.csv')
    income_df = pd.read_csv('../data/preprocessed/barcelona/2018-2025_atles_renda_bruta_llar.csv')
    studies_df = pd.read_csv('../data/preprocessed/barcelona/2018-2025_pad_mdbas_niv-educa-esta_sexe.csv')
    age_df = pd.read_csv('../data/preprocessed/barcelona/2018-2025-pad_mdba_sexe_edat-1.csv')
    neighborhoods_gdf = gpd.read_file('../data/preprocessed/barcelona/barcelona_neighborhoods.geojson')
except FileNotFoundError as e:
    print(f"Error loading data: {e}. Please check your file paths.")
    exit(1)

# Initial Inspection
print("Property Values Info:")
property_values_df.info()
print("\nIncome Info:")
income_df.info()
print("\nStudies Info:")
studies_df.info()
print("\nAge and Population Distribution Info:")
age_df.info()
print("\nNeighborhoods GeoDataFrame Info:")
neighborhoods_gdf.info()

# Display first few rows to understand the content
print("\nProperty Values Head:")
print(property_values_df.head())
print("\nIncome Head:")
print(income_df.head())
print("\nStudies Head:")
print(studies_df.head())
print("\nAge and Population Distribution Head:")
print(age_df.head())
print("\nNeighborhoods Head:")
print(neighborhoods_gdf.head())



Property Values Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51264 entries, 0 to 51263
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Codi_districte  51264 non-null  int64  
 1   Nom_districte   51264 non-null  object 
 2   Codi_barri      51264 non-null  int64  
 3   Nom_barri       51264 non-null  object 
 4   Seccio_censal   51264 non-null  int64  
 5   Desc_valors     51264 non-null  object 
 6   Valors          51264 non-null  float64
 7   year            51264 non-null  int64  
dtypes: float64(1), int64(4), object(3)
memory usage: 3.1+ MB

Income Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9612 entries, 0 to 9611
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Codi_Districte        9612 non-null   int64 
 1   Nom_Districte         9612 non-null   object
 2   Codi_Barri            9612 non-nu

In [19]:
# show unique values in the column "Desc_valors" of property_values_df
print(property_values_df['Desc_valors'].unique())

['Valor_cadastral_total_€' 'Valor_sòl_total_€'
 'Valor_construcció_més_serveis_total_€' 'Valor_cadastral_unitari_€/m2'
 'Valor_sòl_unitari_€/m2' 'Valor_construcció_més_serveis_unitari_€/m2']


# Standardize Column Names & Clean 'Valor' Columns

In [20]:
def standardize_and_clean(df, valor_col='Valor'):
    # Standardize neighborhood and district codes/names
    df.rename(columns={
        'Codi_Districte': 'Codi_districte', 'Nom_Districte': 'Nom_districte',
        'Codi_Barri': 'Codi_barri', 'Nom_Barri': 'Nom_barri'
    }, inplace=True)
    # Convert 'Valor' from object to numeric, treating errors as NaN and filling with 0
    if valor_col in df.columns and df[valor_col].dtype == 'object':
        df[valor_col] = pd.to_numeric(df[valor_col], errors='coerce').fillna(0)
    return df

property_values_df = standardize_and_clean(property_values_df, valor_col='Valors')
income_df = standardize_and_clean(income_df, valor_col='Import_Renda_Bruta_€')
studies_df = standardize_and_clean(studies_df)
age_df = standardize_and_clean(age_df)




# Process Each Feature According to the Data Model


In [ ]:

# A. Median Property Price (from total cadastral value)
prop_price_total = property_values_df[property_values_df['Desc_valors'] == 'Valor_cadastral_total_€']
median_property_price = prop_price_total.groupby(['Codi_barri', 'year'])['Valors'].median().reset_index()
median_property_price.rename(columns={'Valors': 'median_property_price'}, inplace=True)

# B. Average Price per m2 (from unitary cadastral value)
prop_price_m2 = property_values_df[property_values_df['Desc_valors'] == 'Valor_cadastral_unitari_€/m2']
avg_price_per_m2 = prop_price_m2.groupby(['Codi_barri', 'year'])['Valors'].median().reset_index()
avg_price_per_m2.rename(columns={'Valors': 'avg_price_per_m2'}, inplace=True)

# C. Median Household Income
median_household_income = income_df.groupby(['Codi_barri', 'year'])['Import_Renda_Bruta_€'].median().reset_index()
median_household_income.rename(columns={'Import_Renda_Bruta_€': 'median_household_income'}, inplace=True)

# D. Percentage Higher Education
# First, get total population from the studies dataset
total_pop_studies = studies_df.groupby(['Codi_barri', 'year'])['Valor'].sum().reset_index()
total_pop_studies.rename(columns={'Valor': 'total_population_studies'}, inplace=True)
# Then, get population with higher education (levels 5 and 6)
higher_edu_pop = studies_df[studies_df['NIV_EDUCA_esta'].isin([5, 6])]
higher_edu_counts = higher_edu_pop.groupby(['Codi_barri', 'year'])['Valor'].sum().reset_index()
higher_edu_counts.rename(columns={'Valor': 'population_higher_education'}, inplace=True)
# Merge and calculate percentage
education_features = pd.merge(total_pop_studies, higher_edu_counts, on=['Codi_barri', 'year'])
education_features['pct_higher_education'] = (education_features['population_higher_education'] / education_features['total_population_studies']) * 100

# E. Population Density and Percentage of Young Adults (from the Age dataset)
# Get total population
total_population = age_df.groupby(['Codi_barri', 'year'])['Valor'].sum().reset_index()
total_population.rename(columns={'Valor': 'total_population'}, inplace=True)
# Get young adult population (ages 20-39 inclusive)
young_adults_pop = age_df[age_df['EDAT_1'].between(20, 39)]
young_adult_counts = young_adults_pop.groupby(['Codi_barri', 'year'])['Valor'].sum().reset_index()
young_adult_counts.rename(columns={'Valor': 'young_adult_population'}, inplace=True)
# Merge population features together
population_features = pd.merge(total_population, young_adult_counts, on=['Codi_barri', 'year'])
# Calculate pct_young_adults
population_features['pct_young_adults'] = (population_features['young_adult_population'] / population_features['total_population']) * 100



# Handle Geospatial Data (for population density)


In [26]:
# Ensure the GeoDataFrame uses the correct neighborhood code ('BARRI') and is integer type
neighborhoods_gdf.rename(columns={'BARRI': 'Codi_barri'}, inplace=True)
print(neighborhoods_gdf.head())
# Add this code before the line that causes the error to inspect the column
print("Unique values in the 'BARRI' column:")
print(neighborhoods_gdf['BARRI'].unique())
neighborhoods_gdf['Codi_barri'] = neighborhoods_gdf['Codi_barri'].astype(int)
# Project to a suitable CRS for area calculation (e.g., EPSG:25831 for Spain)
neighborhoods_gdf_proj = neighborhoods_gdf.to_crs(epsg=25831)
neighborhoods_gdf['area_km2'] = neighborhoods_gdf_proj.geometry.area / 1_000_000
# Merge population data with area to calculate density
population_features = pd.merge(population_features, neighborhoods_gdf[['Codi_barri', 'area_km2']], on='Codi_barri')
population_features['population_density'] = population_features['total_population'] / population_features['area_km2']


  ID_ANNEX ANNEXDESCR ID_TEMA               TEMA_DESCR ID_CONJUNT  CONJ_DESCR  \
0       01   Grup - I    0104  Unitats Administratives     010417  Gran barri   
1       01   Grup - I    0104  Unitats Administratives     010417  Gran barri   
2       01   Grup - I    0104  Unitats Administratives     010417  Gran barri   
3       01   Grup - I    0104  Unitats Administratives     010417  Gran barri   
4       01   Grup - I    0104  Unitats Administratives     010417  Gran barri   

  ID_SUBCONJ  SCONJ_DESC  ID_ELEMENT         ELEM_DESCR  ... ESTIL_ID  \
0   01041701  Gran barri  0104170101  Límits gran barri  ...        0   
1   01041701  Gran barri  0104170101  Límits gran barri  ...        0   
2   01041701  Gran barri  0104170101  Límits gran barri  ...        0   
3   01041701  Gran barri  0104170101  Límits gran barri  ...        0   
4   01041701  Gran barri  0104170101  Límits gran barri  ...        0   

  ESTIL_QGIS VALOR1QGIS VALOR2QGIS COL_FARCIT FCOL_DESCR FHEX_COLOR COL_DE

KeyError: 'BARRI'

# Assemble the Master DataFrame 


In [ ]:


# List all your feature DataFrames
feature_dfs = [
    median_property_price,
    avg_price_per_m2,
    median_household_income,
    education_features[['Codi_barri', 'year', 'pct_higher_education']],
    population_features[['Codi_barri', 'year', 'population_density', 'pct_young_adults']]
]

# Iteratively merge them together
from functools import reduce
master_df = reduce(lambda left, right: pd.merge(left, right, on=['Codi_barri', 'year'], how='outer'), feature_dfs)

# Add identifiers and geometry
master_df['city'] = 'Barcelona'
# Merge with neighborhood names and geometry from the GeoDataFrame
master_df = pd.merge(master_df, neighborhoods_gdf[['Codi_barri', 'NDESCR_CA', 'geometry']], on='Codi_barri', how='left')
master_df.rename(columns={'NDESCR_CA': 'neighborhood_name', 'Codi_barri': 'neighborhood_id'}, inplace=True)
master_df['neighborhood_id'] = 'BCN_' + master_df['neighborhood_id'].astype(str)

# Display the result
print("--- Master DataFrame Info ---")
master_df.info()
print("\n--- Master DataFrame Head ---")
print(master_df.head())